Цель данного ноутбука: парсинг данных из сохраненных файлов-ответов

In [1]:
# импорт библиотек
import pandas as pd
from bs4 import BeautifulSoup
import re
import glob
import os
from tqdm.auto import tqdm, trange

# импорт собственной функции-парсера (get_data)
# пришлось так сделать, потому как функция большая
# и работа в ноутбуке с функцией и самим парсером
# была неудобной и ноутбук получался большой.
from parse_data import convert_date, get_data

In [3]:
# путь до папки с сохраненными файлами-ответами
path = './data/train_all/data/parsing_all/'

Теперь напишем функцию пробега по файлам и вытаскивания данных

In [59]:
# предварительная функция,
# нужна для создания списка признаков
# исходя из текущей версии функции get_data (можно добавлять/убавлять признаки)
# при таком подходе, поменял функцию get_data, а дальше ничего менять не надо.
def get_col_names():
    # считываем файл и конвертируем его в soup
    with open(path + "all_train_33863740_ok.txt", encoding='utf-8') as file:
        soup = BeautifulSoup(file, "html.parser")

    # парсим его функцием и получаем список актуальных признаков
    data = get_data(soup)
    cols = list(data.keys())

    return cols

# функция собственно перебора файлов и получение финального dataset-а с данными
# входной параметр - список признаков в финальном датасете


def get_all_value(cols):
    # создаем пустой датасет
    data = pd.DataFrame(columns=[cols])

    # создаем список файлов для парсинга
    mask = './data/train_all/data/parsing_all/'
    files = glob.glob(os.path.join(mask, "*.txt"))

    # создаем статус бар для понимания статуса парсинга
    pbar_links = tqdm(range(0, len(files)))
#     pbar_links=tqdm(range(0,10)) # это для отладки кода, пробегаем только 10 файлов

    # теперь идем циклом по списку файлов
    for file in pbar_links:
        pbar_links.set_description("Парсим файл " + str(file+1))

        # получаем имя файла для работы с ним
        file = files[file]

        # считываем файл и конвертируем его в soup
        with open(file, encoding='utf-8') as file_open:
            soup = BeautifulSoup(file_open, "html.parser")

        # извлекаем данные
        df = get_data(soup)

        # далее конвертируем формат данных (здесь наверняка можно сделать элегантнее)
        row_num = len(data)
        value = []
        for col in cols:
            value.append(str(df[col]))

        # и записываем в даасет
        data.loc[row_num] = value

    return data

Собственно парсинг из файлов состоит из двух шагов:
1. Получаем список признаков
2. Парсим все файлы

In [56]:
cols = get_col_names()
dg = get_all_value(cols)

In [57]:
# проверяем итоговый результат
dg.sample(10).T

,90204,81354,89375,36617,19930,36233,27168,53772,46836,70979
bodyType,внедорожник 5 дв.,внедорожник 5 дв.,хэтчбек 5 дв.,внедорожник 5 дв.,седан,седан,лифтбек,лифтбек,седан,лифтбек
brand,VOLKSWAGEN,NISSAN,NISSAN,AUDI,AUDI,BMW,SKODA,SKODA,MERCEDES-BENZ,SKODA
car_url,https://auto.ru/cars/used/sale/volkswagen/taos...,https://auto.ru/cars/used/sale/nissan/qashqai/...,https://auto.ru/cars/used/sale/nissan/march/11...,https://auto.ru/cars/used/sale/audi/q7/1105567...,https://auto.ru/cars/used/sale/audi/a6/1105038...,https://auto.ru/cars/used/sale/bmw/3er/1105561...,https://auto.ru/cars/used/sale/skoda/octavia/1...,https://auto.ru/cars/skoda/rapid/used/,https://auto.ru/cars/used/sale/mercedes/c_klas...,https://auto.ru/cars/used/sale/skoda/octavia/1...
color,серый,чёрный,зелёный,чёрный,чёрный,белый,белый,серебристый,серебристый,чёрный
complectation_dict,"{""id"":""22962515"",""name"":""Status"",""available_op...","{""id"":""0""}","{""id"":""0""}","{""id"":""0""}","{""id"":""0""}","{""id"":""21035350"",""name"":""320d Sport Line"",""ava...","{""id"":""0""}",nan,"{""id"":""0""}","{""id"":""0""}"
description,Комплектация Status:2 USB-C разъема в централь...,"КЛЮЧАВТО | Автомобили с пробегом, направление ...","Продается хорошенький Марч, кузов в порядке, е...","Машина после полной переборки двигателя, было ...",Осмотр г. ТоржокОтличное техническое состояние...,Состояние авто хорошее. Машина мечта.Автомобил...,Машина в нашей семье с 2012 года. Две записи в...,"Шкода рапид в отличном состоянии, пройдено все...","машина в хорошем состоянии,все вопросы по теле...","Автомобиль полностью обслужен, сел и поехал. П..."
engineDisplacement,1395,1997,1275,3597,1984,1995,1390,1598,1998,1798
enginePower,150,144,75,280,180,190,122,105,163,180
equipment_dict,"{""cruise-control"":true,""asr"":true,""esp"":true,""...",{},"{""alarm"":true,""spare-wheel"":true}","{""body-kit"":true,""music-super"":true,""panorama-...",{},"{""esp"":true,""airbag-driver"":true,""sport-seats""...",{},"{""audiosystem-cd"":true,""eco-leather"":true}",{},"{""cruise-control"":true,""asr"":true,""tinted-glas..."
fuelType,бензин,бензин,бензин,бензин,бензин,дизель,бензин,бензин,бензин,бензин


In [58]:
# и сохраняем результат в csv
dg.to_csv('all_auto_ru.csv', encoding='utf-8', index=False)